In [2]:
import numpy as np
import matplotlib.pyplot as plt
import csv
from tabulate import tabulate
np.set_printoptions(precision=6) #Mostrar numeros con maximo seis digitos de precision
np.set_printoptions(suppress=True) # Suprimir uso de notacion cientifica para numeros muy pequenos

In [3]:
N = 23 # Numero de periodos
mYah = 3 # Numero de acciones extraidas de Yahoo
mFin = 4 # Numero de acciones extraidas de Finance.com
m = mYah + mFin # Numero de acciones

precios = np.zeros((m, N+1))
retornos = np.zeros((m, N))

Extrayendo columnas de los archivos de Yahoo Finance:

In [4]:
archivosYahoo = ['csvs/AVH.csv', 'csvs/CIB.csv', 'csvs/EC.csv']

In [5]:
for k in range(mYah):
    archivo = archivosYahoo[k]
    lector = csv.reader(open(archivo))
    lector.__next__() #Ignorar primer renglon
    
    for i in range(N+1):
        precios[k, i] = lector.__next__()[5] #Extraer la 5ta columna: precio de cierre ajustado
        if (i > 0):
            retornos[k, i-1] = (precios[k, i] - precios[k, i-1])/precios[k, i-1]

Extrayendo columnas de los archivos de Investing.com

In [6]:
 archivosFin = ['csvs/ARG-2.csv', 'csvs/CFV-2.csv', 'csvs/FTSE-2.csv', 'csvs/IMI-2.csv']

In [7]:
for k in range(mYah, mYah + mFin):
    archivo = archivosFin[k - mYah]
    lector = csv.reader(open(archivo))
    lector.__next__() #Ignorar primer renglon
    
    for i in range(N, -1, -1):
        precios[k, i] = lector.__next__()[1] #Extraer la 1era columna: precio de cierre
        if (i < N):
            retornos[k, i] = (precios[k, i+1] - precios[k, i])/precios[k, i]

In [8]:
np.shape(retornos)

(7, 23)

TODO: Explicar, a la luz de la lectura 1, la naturaleza de esos activos.

# 3

Vector de Rendimientos promedio

In [9]:
retProm = np.mean(retornos, 1, keepdims = True) # Hallar el promedio de la matriz de retornos a lo largo del eje temporal
print("Tamaño de matriz de rentabilidades:", np.shape(retProm)) # En efecto el vector retProm tiene tamaño 7x1, donde 7 es el número de activos
print("Retornos promedio:\n", retProm)

Tamaño de matriz de rentabilidades: (7, 1)
Retornos promedio:
 [[-0.026526]
 [ 0.018164]
 [ 0.031721]
 [-0.003375]
 [ 0.001484]
 [ 0.010866]
 [ 0.005016]]


Matriz de Covarianzas

In [10]:
S = np.zeros((m, m)) # Inicializacion en 0's

for k in range(m): # Iterar con k sobre activos
    for l in range(m): # Iterar con l sobre activos
        for i in range(N): # Iterar sobre el tiempo con i
            # Para la combinacion de activos k y l se suma la contribución a la covarianza por el tiempo i
            S[k, l] += (retornos[k, i] - retProm[k])*(retornos[l, i] - retProm[l]) 
            
print("Matriz de covarianzas:\n", S)

Matriz de covarianzas:
 [[ 0.305998  0.057651  0.161875 -0.027009  0.058587  0.028235  0.054256]
 [ 0.057651  0.132956  0.126677  0.05594   0.081049  0.068264  0.074749]
 [ 0.161875  0.126677  0.396993 -0.016269  0.089865  0.105785  0.088195]
 [-0.027009  0.05594  -0.016269  0.068726  0.026786  0.022185  0.028931]
 [ 0.058587  0.081049  0.089865  0.026786  0.195841  0.069729  0.070606]
 [ 0.028235  0.068264  0.105785  0.022185  0.069729  0.053816  0.042587]
 [ 0.054256  0.074749  0.088195  0.028931  0.070606  0.042587  0.083977]]


## Matriz de Covarianzas es Definida Positiva

### Matriz Cuadrada

In [11]:
np.shape(S)

(7, 7)

### Simétrica

In [12]:
S - S.transpose() # Debería ser igual a su transpuesta,  y lo es pues su diferencia es 0

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

### Definida Positiva

Usamos la caracterización vista en clase: los subdeterminantes en línea son positivos. En particular vemos que el determinante de la matriz es distinto de $0$.

In [13]:
for k in range(1,m):
    submatPpal = S[:k, :k]
    detp = np.linalg.det(submatPpal)
    print("El determinante de la submatriz  de tamaño ", np.shape(submatPpal), " que contiene la entrada 1,1, es decir la matriz\n", submatPpal, " es ", detp, "!= 0\n")

El determinante de la submatriz  de tamaño  (1, 1)  que contiene la entrada 1,1, es decir la matriz
 [[0.305998]]  es  0.3059979002561973 != 0

El determinante de la submatriz  de tamaño  (2, 2)  que contiene la entrada 1,1, es decir la matriz
 [[0.305998 0.057651]
 [0.057651 0.132956]]  es  0.037360516087251584 != 0

El determinante de la submatriz  de tamaño  (3, 3)  que contiene la entrada 1,1, es decir la matriz
 [[0.305998 0.057651 0.161875]
 [0.057651 0.132956 0.126677]
 [0.161875 0.126677 0.396993]]  es  0.008801954520245776 != 0

El determinante de la submatriz  de tamaño  (4, 4)  que contiene la entrada 1,1, es decir la matriz
 [[ 0.305998  0.057651  0.161875 -0.027009]
 [ 0.057651  0.132956  0.126677  0.05594 ]
 [ 0.161875  0.126677  0.396993 -0.016269]
 [-0.027009  0.05594  -0.016269  0.068726]]  es  0.0002218197081601298 != 0

El determinante de la submatriz  de tamaño  (5, 5)  que contiene la entrada 1,1, es decir la matriz
 [[ 0.305998  0.057651  0.161875 -0.027009  0.058

In [14]:
Sinv = np.linalg.inv(S)
print(Sinv)

[[  5.637903  -4.836339  -3.056531   2.978238  -2.656259  12.325697
   -1.171077]
 [ -4.836339  42.247891  -1.046555 -21.688876   4.678311 -37.582743
  -10.783565]
 [ -3.056531  -1.046555  12.63798   12.130755   6.97623  -33.070479
   -3.639956]
 [  2.978238 -21.688876  12.130755  42.146813   4.804418 -17.507591
   -5.039423]
 [ -2.656259   4.678311   6.97623    4.804418  14.50841  -33.965781
   -6.403059]
 [ 12.325697 -37.582743 -33.070479 -17.507591 -33.965781 168.69121
    9.261645]
 [ -1.171077 -10.783565  -3.639956  -5.039423  -6.403059   9.261645
   28.508659]]


In [15]:
S.dot(Sinv)

array([[ 1.,  0., -0.,  0., -0.,  0.,  0.],
       [-0.,  1., -0., -0.,  0.,  0.,  0.],
       [-0., -0.,  1.,  0., -0., -0.,  0.],
       [-0., -0.,  0.,  1.,  0., -0.,  0.],
       [-0.,  0., -0.,  0.,  1., -0.,  0.],
       [-0.,  0., -0., -0., -0.,  1., -0.],
       [-0.,  0., -0., -0.,  0., -0.,  1.]])

# 4

Parámetros de la teoría

In [16]:
u = np.ones((m, 1))

A = u.transpose().dot(Sinv.dot(u))[0,0]
B = u.transpose().dot(Sinv.dot(retProm))[0,0]
C = retProm.transpose().dot(Sinv.dot(retProm))[0,0]
D = A*C - B**2

print("A =", A, "\t B =", B, "\t C =", C, ", entonces D =", D)

A = 55.79298136178339 	 B = -0.3429301179306104 	 C = 0.01575628695843826 , entonces D = 0.7614891588191541


# 5

Ecuación general de los portafolios óptimos dados los parámetros de la teoría $A, B, C$ y el parámetro $\mu$: $x*(\mu) = (\frac{C - B\mu}{D}) S^{-1} \hat u + (\frac{A \mu - B}{D})S^{-1}\bar r$, en nuestro caso, como:

In [49]:
def xOptMu(mu):
    return ((C - B*mu)/D) * Sinv.dot(u) + ((A*mu - B)/D) * Sinv.dot(retProm)

rentDeseadas = np.linspace(0., 0.4, 9).reshape((9,1,1))

portOptMu = np.zeros( (len(rentDeseadas), m, 1) )

for i in range(len(rentDeseadas)):
    portOptMu[i] = xOptMu(rentDeseadas[i])
    
print(tabulate(np.concatenate((rentDeseadas,portOptMu), axis = 1), ["Mu"] +["% Activo "+str(i) for i in range(1, 8)]))

  Mu    % Activo 1    % Activo 2    % Activo 3    % Activo 4    % Activo 5    % Activo 6    % Activo 7
----  ------------  ------------  ------------  ------------  ------------  ------------  ------------
0        0.0916008     -0.38406      -0.162978      0.171202     -0.258113       1.33337      0.208974
0.05    -0.507786       0.721711     -0.166545     -1.03494      -0.599754       2.24332      0.343998
0.1     -1.10717        1.82748      -0.170113     -2.24109      -0.941394       3.15326      0.479022
0.15    -1.70656        2.93325      -0.173681     -3.44723      -1.28303        4.06321      0.614046
0.2     -2.30594        4.03902      -0.177248     -4.65338      -1.62467        4.97315      0.74907
0.25    -2.90533        5.14479      -0.180816     -5.85952      -1.96632        5.8831       0.884094
0.3     -3.50472        6.25056      -0.184383     -7.06567      -2.30796        6.79305      1.01912
0.35    -4.1041         7.35633      -0.187951     -8.27181      -2.6496   